In [1]:
import os
import h5py
import numpy as np
import scipy
from scipy import integrate
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
import multiprocessing


# EuCd2As2 and EuCd2P2
## Band plot

In [2]:
file_name = 'EuCd2As2_Aafmb_dZ_U5_SOC_scf2_kpdos.dat'
E_F = 2.41394570

#file_name = 'EuCd2P2_Aafmb_dZ_U5_SOC_scf2_kpdos.dat'
#E_F = 3.53706239 - 0.14617176

f = open(file_name, 'r')
content = f.readlines()

k_pts_num = int(content[1].split()[3])
band_num = int(content[1].split()[7])
ion_num = int(content[1].split()[11])

k_pts_list = []; energy_list = []
orbital_component_1 = []; orbital_component_2 = []; orbital_component_3 = []; orbital_component_4 = []

energy_list_temp = []
orbital_component_1_temp = []; orbital_component_2_temp = []
orbital_component_3_temp = []; orbital_component_4_temp = []

line_idx = 2
while line_idx <len(content):
    current_line = content[line_idx].split()
    
    # empty line
    if len(current_line) == 0:
        line_idx += 1
    
    # new k-point
    elif current_line[0] == 'k-point':
        if len(energy_list_temp) > 0:
            energy_list.append(energy_list_temp)
        if len(orbital_component_1_temp) > 0:
            orbital_component_1.append(orbital_component_1_temp)
            orbital_component_2.append(orbital_component_2_temp)
            orbital_component_3.append(orbital_component_3_temp)
            orbital_component_4.append(orbital_component_4_temp)
        k_pts_list.append([float(current_line[3]), float(current_line[4]), float(current_line[5])])
        energy_list_temp = []
        orbital_component_1_temp = []; orbital_component_2_temp = []
        orbital_component_3_temp = []; orbital_component_4_temp = []
        line_idx += 1
        
    # new band at current k-point
    elif current_line[0] == 'band':
        energy_list_temp.append(float(current_line[4])-E_F)
        line_idx += 1
        
    elif current_line[0] == 'ion':
        # load first block
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_1_temp.append(np.array(temp_ar))
        
        # load second block
        line_idx += (ion_num + 2)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_2_temp.append(np.array(temp_ar))
        
        # load third block
        line_idx += (ion_num + 2)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_3_temp.append(np.array(temp_ar))
        
        # load fourth block
        line_idx += (ion_num + 2)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_4_temp.append(np.array(temp_ar))
        
        line_idx += (ion_num + 2)
        
    else:
        line_idx += 1
        
# laod last k-point
energy_list.append(energy_list_temp)
orbital_component_1.append(orbital_component_1_temp)
orbital_component_2.append(orbital_component_2_temp)
orbital_component_3.append(orbital_component_3_temp)
orbital_component_4.append(orbital_component_4_temp)

f.close()

k_pts_list = np.array(k_pts_list)
energy_list = np.array(energy_list)


In [ ]:
k_list_plot = np.arange(0, 1400, 1)

# Uniform length of high symmetry path
#k_path = k_list_plot

# EuCd2As2
k_path = list(np.linspace(0, 1.643/2, 200))\
+list(np.linspace(1.643/2, 1.643/2+1.643/4, 200))\
+list(np.linspace(1.643/2+1.643/4, 1.643/2+1.643/4+0.9434, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434, 1.643/2+1.643/4+0.9434+0.8574/2, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434+0.8574/2, 1.643/2+1.643/4+0.9434+0.8574/2+1.643/2, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434+0.8574/2+1.643/2, 1.643/2+1.643/4+0.9434+0.8574/2+1.643/2+1.643/4, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434+0.8574/2+1.643/2+1.643/4, 1.643/2+1.643/4+0.9434+0.8574/2+1.643/2+1.643/4+0.9434, 200))

# EuCd2P2
'''
k_path = list(np.linspace(0, 1.677/2, 200))\
+list(np.linspace(1.677/2, 1.677/2+1.677/4, 200))\
+list(np.linspace(1.677/2+1.677/4, 1.677/2+1.677/4+0.9682, 200))\
+list(np.linspace(1.677/2+1.677/4+0.9682, 1.677/2+1.677/4+0.9682+0.8752/2, 200))\
+list(np.linspace(1.677/2+1.677/4+0.9682+0.8752/2, 1.677/2+1.677/4+0.9682+0.8752/2+1.677/2, 200))\
+list(np.linspace(1.677/2+1.677/4+0.9682+0.8752/2+1.677/2, 1.677/2+1.677/4+0.9682+0.8752/2+1.677/2+1.677/4, 200))\
+list(np.linspace(1.677/2+1.677/4+0.9434+0.8752/2+1.677/2+1.677/4, 1.677/2+1.677/4+0.9434+0.8752/2+1.677/2+1.677/4+0.9682, 200))
'''
plt.figure(figsize=(12,9))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_path, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        #Eu_5d = np.sum(orbital_component[0:2, 4:9])
        Cd_5s = np.sum(orbital_component[2:6, 0])
        Eu_4f = np.sum(orbital_component[0:2, 9:16])
        As_4p = np.sum(orbital_component[6:10, 1:4])
        return [k_path[j], energy_list[j,k], [Cd_5s, Eu_4f, As_4p]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])
    
dot_size_ratio = 20
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='red', alpha=.5, label='Cd 5s')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='orange', alpha=.5, label='Eu 4f')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][2] for j in range(len(cache))], c='blue', alpha=.5, label='As 4p')

#plt.grid(which='both' ,axis = 'x')
#plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-3, 2)
plt.xlim(k_path[0], k_path[-1])
#plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.pdf']), dpi=100)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.png']), dpi=100)


In [ ]:
k_list_plot = np.arange(400, 800, 1)

# Uniform length of high symmetry path
#k_path = k_list_plot

# EuCd2As2
'''
k_path = list(np.linspace(1.643/2+1.643/4, 1.643/2+1.643/4+0.9434, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434, 1.643/2+1.643/4+0.9434+0.8574/2, 200))
'''

# EuCd2P2
'''
k_path = list(np.linspace(1.677/2+1.677/4, 1.677/2+1.677/4+0.9682, 200))\
+list(np.linspace(1.677/2+1.677/4+0.9682, 1.677/2+1.677/4+0.9682+0.8752/2, 200))
'''

k_path = list(np.linspace(1.643/2+1.643/4, 1.643/2+1.643/4+0.9434, 200))\
+list(np.linspace(1.643/2+1.643/4+0.9434, 1.643/2+1.643/4+0.9434+0.8574/2, 200))

plt.figure(figsize=(12,9))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_path, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        #Eu_5d = np.sum(orbital_component[0:2, 4:9])
        Cd_5s = np.sum(orbital_component[2:6, 0])
        Eu_4f = np.sum(orbital_component[0:2, 9:16])
        As_4p = np.sum(orbital_component[6:10, 1:4])
        return [k_path[j-k_list_plot[0]], energy_list[j,k], [Cd_5s, Eu_4f, As_4p]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])
    
dot_size_ratio = 20
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            c=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], cmap='GnBu', alpha=.5, label='Eu 4f')

#plt.grid(which='both' ,axis = 'x')
#plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-1.2, 0.1)
plt.xlim(k_path[0], k_path[-1])
#plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.pdf']), dpi=100)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.png']), dpi=100)


In [ ]:
plt.ylim(-1.2, 0.1)
plt.xlim(0, k_path[-1])
#plt.legend()
plt.savefig('frame.pdf', dpi=100)


## Band plot along high-symmetry
### As/P $p_x$, $p_y$, $p_z$ orbital

In [ ]:
k_list_plot = np.arange(600, 399, -1)
plt.figure(figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], color='black', alpha=.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_compoent = orbital_component_1[j][k]
        As_4py = np.sum(orbital_compoent[6:10, 1])
        As_4pz = np.sum(orbital_compoent[6:10, 2])
        As_4px = np.sum(orbital_compoent[6:10, 3])
    
        return [j, energy_list[j,k], [As_4py, As_4pz, As_4px]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])

dot_size_ratio = 100
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='orange', alpha=1, label = 'P py')
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='orange', alpha=1, label = 'P pz')
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][2] for j in range(len(cache))], c='orange', alpha=1, label = 'P px')

#plt.grid(which='both' ,axis = 'x')
#plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-1, 0.1)
#plt.xlim(k_list_plot[0], k_list_plot[-1])
#plt.legend()
#plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.pdf']), dpi=300, transparent=True)
#plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.png']), dpi=300, transparent=True)
plt.savefig('temp.png', dpi=100)


### orbital component for different atoms

In [ ]:
k_list_plot = np.arange(600, 399, -1)
plt.figure(figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        Cd_5s = np.sum(orbital_component[2:6, 0])
        Eu_4f = np.sum(orbital_component[0:2, 9:16])
        As_4p = np.sum(orbital_component[6:10, 1:4])
        
        return [j, energy_list[j,k], [Cd_5s, Eu_4f, As_4p]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])

dot_size_ratio = 100
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='red', alpha=1, label = 'Cd 5s')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='orange', alpha=1, label = 'Eu 4f')
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][2] for j in range(len(cache))], c='blue', alpha=1, label = 'As 3p')

#plt.grid(which='both' ,axis = 'x')
plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-1, 0.1)
plt.xlim(k_list_plot[0], k_list_plot[-1])
#plt.legend()
#plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.pdf']), dpi=100)
#plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.png']), dpi=100)
plt.savefig('temp.png', dpi=100)


In [ ]:
plt.figure(figsize=(3,6))
for band_idx in range(1):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

plt.ylim(-1, 0.1)
plt.xlim(0, len(k_list_plot))
#plt.legend()
plt.savefig('frame_high_symmetry.pdf', dpi=100)

### orbital component for different band

In [ ]:
k_list_plot = np.arange(600, 399, -1)

plt.figure(1, figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], alpha = 0.3, color = 'black')
    
# for EuCd2As2
band_idx_alpha = [140, 141]; band_idx_beta = [138, 139]; band_idx_gamma = [136, 137]

# for EuCd2P2
#band_idx_alpha = [100, 101]; band_idx_beta = [98, 99]; band_idx_gamma = [96, 97]

cache_alpha = []; cache_beta = []; cache_gamma = []

for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k[0]] + orbital_component_1[j][k[1]]
        #Cd_5s = np.sum(orbital_component[2:6, 0])
        Eu_4f = np.sum(orbital_component[0:2, 9:16])
        As_4p = np.sum(orbital_component[6:10, 1:4])

        return [j, energy_list[j,k[0]], [Eu_4f, As_4p]]
    
    cache_alpha.extend([func(band_idx_alpha)])
    cache_beta.extend([func(band_idx_beta)])
    cache_gamma.extend([func(band_idx_gamma)])


dot_size_ratio = 20
plt.scatter([cache_alpha[j][0] for j in range(len(cache_alpha))], [cache_alpha[j][1] for j in range(len(cache_alpha))], \
            s=[dot_size_ratio*cache_alpha[j][2][0] for j in range(len(cache_alpha))], c='red', alpha=1, label = 'alpha, Eu 4f')
plt.scatter([cache_beta[j][0] for j in range(len(cache_beta))], [cache_beta[j][1] for j in range(len(cache_beta))], \
            s=[dot_size_ratio*cache_beta[j][2][0] for j in range(len(cache_beta))], c='orange', alpha=1, label = 'beta, Eu 4f') 
plt.scatter([cache_gamma[j][0] for j in range(len(cache_gamma))], [cache_gamma[j][1] for j in range(len(cache_gamma))], \
            s=[dot_size_ratio*cache_gamma[j][2][0] for j in range(len(cache_gamma))], c='blue', alpha=1, label = 'gamma, Eu 4f')

plt.xlim(k_list_plot[0], k_list_plot[-1])
plt.ylim(-1.0, 0.2)
plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_1', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_1', '.png']), dpi=300)


plt.figure(2, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][0] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, Eu 4f')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][0] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, Eu 4f')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][0] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, Eu 4f')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.0, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_2', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_2', '.png']), dpi=300)

np.savetxt('alpha_f.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                     [cache_alpha[j][2][0] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_f.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                     [cache_beta[j][2][0] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_f.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                     [cache_gamma[j][2][0] for j in range(len(cache_gamma))]]).T, delimiter=',')



plt.figure(3, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][1] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P p')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][1] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P p')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][1] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P p')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.0, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_3', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_3', '.png']), dpi=300)



In [ ]:
k_list_plot = np.arange(600, 399, -1)

plt.figure(1, figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], alpha = 0.3, color = 'black')

# for EuCd2As2
#band_idx_alpha = [140, 141]; band_idx_beta = [138, 139]; band_idx_gamma = [136, 137]

# for EuCd2P2
band_idx_alpha = [100, 101]; band_idx_beta = [98, 99]; band_idx_gamma = [96, 97]

cache_alpha = []; cache_beta = []; cache_gamma = []

for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k[0]] + orbital_component_1[j][k[1]]
        As_4py = np.sum(orbital_component[6:10, 1])
        As_4pz = np.sum(orbital_component[6:10, 2])
        As_4px = np.sum(orbital_component[6:10, 3])

        return [j, energy_list[j,k[0]], [As_4py, As_4pz, As_4px]]
    
    cache_alpha.extend([func(band_idx_alpha)])
    cache_beta.extend([func(band_idx_beta)])
    cache_gamma.extend([func(band_idx_gamma)])


dot_size_ratio = 20
plt.scatter([cache_alpha[j][0] for j in range(len(cache_alpha))], [cache_alpha[j][1] for j in range(len(cache_alpha))], \
            s=[dot_size_ratio*cache_alpha[j][2][0] for j in range(len(cache_alpha))], c='red', alpha=1, label = 'alpha, P py')
plt.scatter([cache_beta[j][0] for j in range(len(cache_beta))], [cache_beta[j][1] for j in range(len(cache_beta))], \
            s=[dot_size_ratio*cache_beta[j][2][0] for j in range(len(cache_beta))], c='orange', alpha=1, label = 'beta, P pz') 
plt.scatter([cache_gamma[j][0] for j in range(len(cache_gamma))], [cache_gamma[j][1] for j in range(len(cache_gamma))], \
            s=[dot_size_ratio*cache_gamma[j][2][0] for j in range(len(cache_gamma))], c='blue', alpha=1, label = 'gamma, P px')

plt.xlim(k_list_plot[0], k_list_plot[-1])
plt.ylim(-1.0, 0.2)
plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_4', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_4', '.png']), dpi=300)


plt.figure(2, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][0] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P py')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][0] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P py')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][0] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P py')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.0, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_5', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_5', '.png']), dpi=300)

np.savetxt('alpha_py.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][0] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_py.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][0] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_py.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][0] for j in range(len(cache_gamma))]]).T, delimiter=',')


plt.figure(3, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][1] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P pz')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][1] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P pz')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][1] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P pz')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.0, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_6', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_6', '.png']), dpi=300)

np.savetxt('alpha_pz.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][1] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_pz.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][1] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_pz.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][1] for j in range(len(cache_gamma))]]).T, delimiter=',')


plt.figure(4, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][2] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P px')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][2] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P px')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][2] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P px')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.0, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_7', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_7', '.png']), dpi=300)

np.savetxt('alpha_px.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][2] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_px.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][2] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_px.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][2] for j in range(len(cache_gamma))]]).T, delimiter=',')



## density of states

In [ ]:
'''            
DoS list: 
    -- first element: total DoS, first column --> energy, second/third column --> total DoS for spin up/dn
    -- remaining elements: DoS for each atom, first column --> energy, remaining column --> decomposed DoS, with column sequence\
    (s, m_x, m_y, m_z, py, m_x, m_y, m_z, pz, m_x, m_y, m_z, px, m_x, m_y, m_z, d_xy, m_x, m_y, m_z, dyz, m_x, m_y, m_z,\
    dz2, m_x, m_y, m_z, dxz, m_x, m_y, m_z, dx2y2, m_x, m_y, m_z, fy3x2, m_x, m_y, m_z, fxyz, m_x, m_y, m_z,\
    fyz2, m_x, m_y, m_z, fz3, m_x, m_y, m_z, fxz2, m_x, m_y, m_z, fzx2, m_x, m_y, m_z, fx3, m_x, m_y, m_z)
    
'''

#file_name = 'EuCd2As2_Aafmb_dZ_U5_SOC_scf2_ldos.dat'
#E_F = 2.41394570

file_name = 'EuCd2P2_Aafmb_dZ_U5_SOC_scf2_ldos.dat'
E_F = 3.53706239

f = open(file_name, 'r')
content = f.readlines()
f.close()

DoS = []; DoS_atom = []

line_idx = 6


while line_idx < len(content):
    current_line = content[line_idx].split()
    
    # new atom
    if len(current_line) == 5:
        DoS.append(np.array(DoS_atom))
        DoS_atom = []
        line_idx += 1
    elif len(current_line) == 0:
        line_idx += 1
    else:
        if len(current_line) == 3:
            DoS_atom.append([float(j) for j in current_line])
            line_idx += 1
            
        else:
            next_line = content[line_idx+1].split()
            DoS_atom.append([float(j) for j in current_line] + [float(j) for j in next_line])
            line_idx += 2
            
DoS.append(np.array(DoS_atom))


In [ ]:
s_orbital = [1]; p_orbital = [5, 9, 13]; d_orbital = [17, 21, 25, 29, 33]; f_orbital = [37, 41, 45, 49, 53, 57, 61]
Eu = [1, 2]; Cd = [3, 4, 5, 6]; As = [7, 8, 9, 10]

energy = DoS[0][:, 0] - E_F
DoS_total = np.sum(DoS[0][:, 1:3], axis=1)
DoS_Eu_f = np.zeros(DoS[0].shape[0])
DoS_As_p = np.zeros(DoS[0].shape[0])

for j in Eu:
    for k in f_orbital:
        DoS_Eu_f += DoS[j][:, k]
    
for j in As:
    for k in p_orbital:
        DoS_As_p += DoS[j][:, k]

plt.figure(figsize=(6,3))

#plt.plot(energy, DoS_total, linestyle = '--', color = 'black', label = 'total')
plt.plot(energy, DoS_Eu_f, label = 'Eu f')
plt.plot(energy, DoS_As_p, label = 'P p')


plt.legend()
plt.xlim(-3, 2)
plt.hlines(0, min(energy), max(energy), color = 'black', linestyle = '--')
plt.xlabel('E-E_F (eV)')
plt.ylabel('Density of states')


# SrCd2P2


In [ ]:
#file_name = 'SrCd2P2_single_SOC_scf1_kpdos.dat'
#E_F = 3.54459008
#E_F = 3.54459008 + 0.4

#file_name = 'SrCd2P2_single_SOC_scf2_kpdos.dat'
#E_F = 3.26176426

file_name = 'SrCd2P2_single_SOC_scf2_kpdos.dat'
E_F = 3.26176426 + .5

f = open(file_name, 'r')
content = f.readlines()

k_pts_num = int(content[1].split()[3])
band_num = int(content[1].split()[7])
ion_num = int(content[1].split()[11])

k_pts_list = []; energy_list = []
orbital_component_1 = []; orbital_component_2 = []; orbital_component_3 = []; orbital_component_4 = []

energy_list_temp = []
orbital_component_1_temp = []; orbital_component_2_temp = []
orbital_component_3_temp = []; orbital_component_4_temp = []

line_idx = 2
while line_idx <len(content):
    current_line = content[line_idx].split()
    
    # empty line
    if len(current_line) == 0:
        line_idx += 1
        
    # new k-point
    elif current_line[0] == 'k-point':
        if len(energy_list_temp) > 0:
            energy_list.append(energy_list_temp)
        if len(orbital_component_1_temp) > 0:
            orbital_component_1.append(orbital_component_1_temp)
            orbital_component_2.append(orbital_component_2_temp)
            orbital_component_3.append(orbital_component_3_temp)
            orbital_component_4.append(orbital_component_4_temp)
        k_pts_list.append([float(current_line[3]), float(current_line[4]), float(current_line[5])])
        energy_list_temp = []
        orbital_component_1_temp = []; orbital_component_2_temp = []
        orbital_component_3_temp = []; orbital_component_4_temp = []
        line_idx += 1
        
    # new band at current k-point
    elif current_line[0] == 'band':
        energy_list_temp.append(float(current_line[4])-E_F)
        line_idx += 1
        
    elif current_line[0] == 'ion':
        # load first block
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_1_temp.append(np.array(temp_ar))
        
        # load second block
        line_idx += (ion_num + 1)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_2_temp.append(np.array(temp_ar))
        
        # load third block
        line_idx += (ion_num + 1)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_3_temp.append(np.array(temp_ar))
        
        # load fourth block
        line_idx += (ion_num + 1)
        temp_ar = []
        for ion_idx in np.arange(1, ion_num+1, 1):
            temp_1 = [float(j) for j in content[line_idx+ion_idx].split()]
            temp_ar.append(temp_1[1:(len(temp_1)-1)])
        orbital_component_4_temp.append(np.array(temp_ar))
        
        line_idx += (ion_num + 1)
        
    else:
        line_idx += 1
        
energy_list.append(energy_list_temp)
orbital_component_1.append(orbital_component_1_temp)
orbital_component_2.append(orbital_component_2_temp)
orbital_component_3.append(orbital_component_3_temp)
orbital_component_4.append(orbital_component_4_temp)

f.close()

k_pts_list = np.array(k_pts_list)
energy_list = np.array(energy_list)


In [ ]:
k_list_plot = np.arange(0, 1200, 1)
#k_list_plot = np.arange(300, 200, -1)
plt.figure(figsize=(12,9))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        Cd_5s = np.sum(orbital_component[1:3, 0])
        As_4p = np.sum(orbital_component[3:5, 1:4])
        return [j, energy_list[j,k], [Cd_5s, As_4p]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])
    
dot_size_ratio = 20
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='red', alpha=.5, label = 'Cd 5s')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='blue', alpha=.5, label = 'As 4p')

plt.grid(which='both' ,axis = 'x')
plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-3, 2)
plt.xlim(0, len(k_list_plot))
plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '.png']), dpi=300)



In [ ]:
k_list_plot = np.arange(400, 600, 1)
plt.figure(figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], c = 'red', alpha=.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        Cd_5s = np.sum(orbital_component[1:3, 0])
        As_4p = np.sum(orbital_component[3:5, 1:4])

        return [j, energy_list[j,k], [Cd_5s, As_4p]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])

dot_size_ratio = 30
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='red', alpha=.5, label = 'Cd 5s')
#plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
#            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='blue', alpha=.5, label = 'P 3p')

#plt.grid(which='both' ,axis = 'x')
plt.hlines(0, 0, len(k_list_plot), color='red', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-2.4, 0.1)
plt.xlim(k_list_plot[0], k_list_plot[-1])
#plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.pdf']), dpi=300, transparent=True)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.png']), dpi=300, transparent=True)


In [ ]:
k_list_plot = np.arange(300, 200, -1)
plt.figure(figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], c = 'black', alpha=0.5)

cache = []
for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k]
        P_py = np.sum(orbital_component[3:5, 1])
        P_pz = np.sum(orbital_component[3:5, 2])
        P_px = np.sum(orbital_component[3:5, 3])
        
        return [j, energy_list[j,k], [P_py, P_pz, P_px]]
    
    cache.extend([func(k) for k in range(energy_list.shape[1])])

dot_size_ratio = 50
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][0] for j in range(len(cache))], c='red', alpha=.5, label = 'P py')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][1] for j in range(len(cache))], c='orange', alpha=.5, label = 'P pz')
plt.scatter([cache[j][0] for j in range(len(cache))], [cache[j][1] for j in range(len(cache))], \
            s=[dot_size_ratio*cache[j][2][2] for j in range(len(cache))], c='blue', alpha=.5, label = 'P px')

#plt.grid(which='both' ,axis = 'x')
plt.hlines(0, 0, len(k_list_plot), color='black', linestyle = '--', alpha=.5)
plt.ylabel('E-E_F (eV)')
plt.ylim(-1.5, 0.1)
plt.xlim(k_list_plot[0], k_list_plot[-1])
plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_GammaK.png']), dpi=300)


In [ ]:
k_list_plot = np.arange(300, 200, -1)

band_idx_alpha = np.arange(37, 40, 1)
band_idx_gamma = [38, 39]
band_idx_beta = [40, 41]
band_idx_alpha = [42, 43]

plt.figure(1, figsize=(3,6))
for band_idx in band_idx_alpha:
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], alpha = 0.3, color = 'black')

plt.ylim(-1.5, 0.1)
plt.xlim(k_list_plot[0], k_list_plot[-1])


In [ ]:
k_list_plot = np.arange(0, 1400, -1)

plt.figure(1, figsize=(3,6))
for band_idx in range(energy_list.shape[1]):
    plt.plot(k_list_plot, energy_list[k_list_plot, band_idx], alpha = 0.3, color = 'black')

# for EuCd2As2
#band_idx_alpha = [140, 141]; band_idx_beta = [138, 139]; band_idx_gamma = [136, 137]

# for EuCd2P2
#band_idx_alpha = [100, 101]; band_idx_beta = [98, 99]; band_idx_gamma = [96, 97]

# for SrCd2P2
band_idx_alpha = [42, 43]; band_idx_beta = [40, 41]; band_idx_gamma = [38, 39]

cache_alpha = []; cache_beta = []; cache_gamma = []

for j in k_list_plot:
    def func(k):
        orbital_component = orbital_component_1[j][k[0]] + orbital_component_1[j][k[1]]
        As_4py = np.sum(orbital_component[3:5, 1])
        As_4pz = np.sum(orbital_component[3:5, 2])
        As_4px = np.sum(orbital_component[3:5, 3])

        return [j, energy_list[j,k[0]], [As_4py, As_4pz, As_4px]]
    
    cache_alpha.extend([func(band_idx_alpha)])
    cache_beta.extend([func(band_idx_beta)])
    cache_gamma.extend([func(band_idx_gamma)])

dot_size_ratio = 20
plt.scatter([cache_alpha[j][0] for j in range(len(cache_alpha))], [cache_alpha[j][1] for j in range(len(cache_alpha))], \
            s=[dot_size_ratio*cache_alpha[j][2][1] for j in range(len(cache_alpha))], c='red', alpha=1, label = 'alpha, P pz')
plt.scatter([cache_beta[j][0] for j in range(len(cache_beta))], [cache_beta[j][1] for j in range(len(cache_beta))], \
            s=[dot_size_ratio*cache_beta[j][2][1] for j in range(len(cache_beta))], c='orange', alpha=1, label = 'beta, P pz') 
plt.scatter([cache_gamma[j][0] for j in range(len(cache_gamma))], [cache_gamma[j][1] for j in range(len(cache_gamma))], \
            s=[dot_size_ratio*cache_gamma[j][2][1] for j in range(len(cache_gamma))], c='blue', alpha=1, label = 'gamma, P pz')

plt.xlim(k_list_plot[0], k_list_plot[-1])
plt.ylim(-1.5, 0.2)
plt.legend()
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_4', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_4', '.png']), dpi=300)


plt.figure(2, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][0] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P py')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][0] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P py')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][0] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P py')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.5, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_5', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_5', '.png']), dpi=300)

np.savetxt('alpha_py.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][0] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_py.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][0] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_py.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][0] for j in range(len(cache_gamma))]]).T, delimiter=',')


plt.figure(3, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][1] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P pz')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][1] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P pz')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][1] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P pz')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.5, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_6', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_6', '.png']), dpi=300)

np.savetxt('alpha_pz.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][1] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_pz.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][1] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_pz.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][1] for j in range(len(cache_gamma))]]).T, delimiter=',')


plt.figure(4, figsize=(6,3))
plt.plot([cache_alpha[j][1] for j in range(len(cache_alpha))], [cache_alpha[j][2][2] for j in range(len(cache_alpha))], c='red', alpha=1, label='alpha, P px')
plt.plot([cache_beta[j][1] for j in range(len(cache_beta))], [cache_beta[j][2][2] for j in range(len(cache_beta))], c='orange', alpha=1, label='beta, P px')
plt.plot([cache_gamma[j][1] for j in range(len(cache_gamma))], [cache_gamma[j][2][2] for j in range(len(cache_gamma))], c='blue', alpha=1, label='gamma, P px')
plt.xlabel('E - $E_F$ (eV)')
plt.ylabel('density of states')
plt.legend()
plt.xlim(-1.5, 0.2)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_7', '.pdf']), dpi=300)
plt.savefig(''.join([file_name[0:(len(file_name)-4)], '_band_project_7', '.png']), dpi=300)

np.savetxt('alpha_px.csv', np.array([[cache_alpha[j][1] for j in range(len(cache_alpha))],  \
                                    [cache_alpha[j][2][2] for j in range(len(cache_alpha))]]).T, delimiter=',')
np.savetxt('beta_px.csv', np.array([[cache_beta[j][1] for j in range(len(cache_beta))],  \
                                    [cache_beta[j][2][2] for j in range(len(cache_beta))]]).T, delimiter=',')
np.savetxt('gamma_px.csv', np.array([[cache_gamma[j][1] for j in range(len(cache_gamma))],  \
                                    [cache_gamma[j][2][2] for j in range(len(cache_gamma))]]).T, delimiter=',')

